In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from src.general.make_noise import beep_on_error, beep_repeat, beep
beep_on_error()

In [ ]:
import pandas as pd
import numpy as np
from src.data_tools.filename_skim_tools import regex_select, get_files, make_df
from src.data_tools.bff_meta import preselection, band_cut, isin, identity, all_reg, jet_variations
from src.data_tools.bff_cuts import *

from glob import glob
import re

In [ ]:
def one_in(list_of_strings, test_string):
    for string_in in list_of_strings:
        if string_in in test_string: return True
    return False

In [ ]:
from src.data_tools.get_file_list import get_file_df
from src.data_tools.get_data import get_data
from src.data_tools.apply_cuts_v2 import  process_sample

In [ ]:
output_dir = 'assets_feb_23'

In [ ]:
era = 2016
file_df = get_file_df(path = "{}/data".format(output_dir))
file_df = file_df[file_df.era==era]
file_df.reset_index(inplace=True)

In [ ]:
##
## hist making
##

In [ ]:
from src.plotting_tools.Bins import Bins, bins, binning_type
from src.plotting_tools.cms_format import cms_format_fig, cms_style
from src.plotting_tools.colors import color_fader
from src.assets.regions import nJets, regions, region_and_label
from src.assets.file_groups import bck_dict, bck_list, bck_colors
from src.plotting_tools.draw_stack_plot_hists import draw_bckground, draw_signals, draw_data, draw_stackplot
from src.plotting_tools.latexAssets import HTLT, RelMET
import matplotlib.pyplot as plt

from src.plotting_tools.SysHist import make_sys_hist, SysHist

from src.assets.lumi import lumi_dict
lumi = lumi_dict[str(era)]

In [ ]:
hists = {
"DiLepMass": (Bins(np.linspace(105, 900, (900-105)+1)), 1), 
"HTLT": (Bins(np.linspace(-800,800,100)), .1), 
"RelMET": (Bins(np.linspace(0, 2,100)), .1), 
"minGoodJetElDR": (Bins(np.linspace(0,5,100)), .1), 
}

In [ ]:
draw_inline = True
if draw_inline:
    plt.ion()
else:
    plt.ioff()

In [ ]:
def make_hist_dict(row, reg, feature):
    bins, select_level = hists[feature]
    isdata = row.type=='data'
    
    hist = make_sys_hist(df, feature, reg, bin_edges=bins.bin_edges,
                     ind_sys_hist=False, select_level=select_level, 
                         isdata=isdata)
    
    hist_dict = hist.to_dict()
    
    hist_dict['reg'] = reg
    hist_dict['feature'] = feature
    if feature=='minGoodJetElDR': print(reg, feature, hist.nominal.sum())
    return {**row.to_dict(), **hist_dict}
    

In [ ]:
hist_list = []
first_iter = 0
for i, row in file_df.iterrows():
    print("-----")
    df = process_sample(row, era, verbose=True, trigger_fix=True)
    for reg in regions:
        for hist in hists:
            hist_list.append(make_hist_dict(row, reg, hist))
            
    del df
    
pd.DataFrame(hist_list).to_pickle('{}/data/combined_{}_flat_hist.pkl'.format(output_dir, era))

In [ ]:
pd.DataFrame(hist_list).to_pickle('{}/data/combined_{}_flat_hist.pkl'.format(output_dir, era))

In [ ]:
beep_repeat()

In [ ]:
test_df = pd.read_pickle('{}/data/combined_{}_flat_hist.pkl'.format(output_dir, era))

In [ ]:
fig, ax = plt.subplots(1)
hist = SysHist.from_dict(test_df.iloc[20].to_dict())
hist.draw(ax)
print(hist.sys_summary())

In [ ]:
fig, ax = plt.subplots(1)
SysHist.from_dict(hist_list[20]).draw(ax)